In [ ]:
!pip install langchain

In [ ]:
!pip install -U langchain-community

In [30]:
%pip install --upgrade --quiet  langchain langchain-google-vertexai "langchain-google-community[featurestore]"

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cloud-tpu-client 0.10 requires google-api-python-client==1.8.0, but you have google-api-python-client 2.147.0 which is incompatible.
tensorflow 2.11.0 requires protobuf<3.20,>=3.9.2, but you have protobuf 3.20.3 which is incompatible.
tensorflow-serving-api 2.11.0 requires protobuf<3.20,>=3.9.2, but you have protobuf 3.20.3 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [4]:
from google.cloud import aiplatform
import google.cloud.bigquery as bq
import langchain
from langchain.llms import VertexAI
from langchain.prompts import PromptTemplate
from langchain.schema import format_document
from langchain_community.document_loaders import DirectoryLoader, TextLoader,BigQueryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_google_vertexai import VertexAIEmbeddings
from langchain_google_community import BigQueryVectorStore

import uuid

In [13]:
# Define our query
query = """
SELECT id,media_type,content,test_metadata 
FROM `nine-quality-test.Nine_Quality_Test.content_embeddings` ;
"""

# Load the data
loader = BigQueryLoader(
    query, metadata_columns=["id"], page_content_columns=["content","media_type","test_metadata"]
)
data = loader.load()

In [8]:
docs = []
docs.extend(loader.load())
text_splitter = RecursiveCharacterTextSplitter(chunk_size=20, chunk_overlap  = 5)
docs = text_splitter.split_documents(docs)

In [9]:
doc_ids = [str(uuid.uuid4()) for _ in docs]
chunked_docs=[]
for i, doc in enumerate(docs):
    _id = doc_ids[i]
    doc.metadata['chunk_id']=_id

chunked_docs.extend(docs)

In [11]:
PROJECT = !gcloud config get-value project
PROJECT_ID = PROJECT[0]
REGION = "us-central1"

In [12]:
DATASET = "my_langchain_dataset"  # @param {type: "string"}
TABLE = "doc_and_vectors"  # @param {type: "string"}


embedding = VertexAIEmbeddings(
    model_name="textembedding-gecko@latest", project=PROJECT_ID
)


store = BigQueryVectorStore(
    project_id=PROJECT_ID,
    dataset_name=DATASET,
    table_name=TABLE,
    location=REGION,
    embedding=embedding,
)

BigQuery table nine-quality-test.my_langchain_dataset.doc_and_vectors initialized/validated as persistent storage. Access via BigQuery console:
 https://console.cloud.google.com/bigquery?project=nine-quality-test&ws=!1m5!1m4!4m3!1snine-quality-test!2smy_langchain_dataset!3sdoc_and_vectors


In [ ]:
retriever = MultiVectorRetriever(
    vectorstore=vectorstore,
    byte_store=store,
    id_key=id_key,
)

### source

https://cloud.google.com/blog/products/ai-machine-learning/rag-with-bigquery-and-langchain-in-cloud

In [33]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

documents = []
documents.extend(loader.load())
 
# Split the documents into chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=20,
    chunk_overlap=5#,
    #separators=["\n\n", "\n", ".", "!", "?", ",", " ", ""],
)
doc_splits = text_splitter.split_documents(documents)

# Add chunk number to metadata
chunk_idx=0
prev=doc_splits[0].metadata["id"]
for idx, split in enumerate(doc_splits):
    if prev==split.metadata["id"]:
       split.metadata["chunk"] = chunk_idx
      
    else:
        chunk_idx=0
        split.metadata["chunk"] = chunk_idx
        prev=split.metadata["id"]
    chunk_idx +=1
        

In [ ]:
from langchain_google_vertexai import VertexAIEmbeddings
from langchain_google_community import BigQueryVectorStore


embedding_model = VertexAIEmbeddings(
    model_name="textembedding-gecko@latest", project=PROJECT_ID
)
bq_store = BigQueryVectorStore(
    project_id=PROJECT_ID,
    dataset_name=DATASET,
    table_name=TABLE,
    location=REGION,
    embedding=embedding,
) 

bq_store.add_documents(doc_splits)

In [30]:
doc_splits

[Document(metadata={'id': 'tr945181.png', 'chunk': 0}, page_content='media_type: image'),
 Document(metadata={'id': 'tr945181.png', 'chunk': 1}, page_content='content:  The image'),
 Document(metadata={'id': 'tr945181.png', 'chunk': 2}, page_content='is a top-down view'),
 Document(metadata={'id': 'tr945181.png', 'chunk': 3}, page_content='view of a camel'),
 Document(metadata={'id': 'tr945181.png', 'chunk': 4}, page_content='walking in the'),
 Document(metadata={'id': 'tr945181.png', 'chunk': 5}, page_content='the desert'),
 Document(metadata={'id': 'tr945181.png', 'chunk': 6}, page_content='. The camel is in'),
 Document(metadata={'id': 'tr945181.png', 'chunk': 7}, page_content='in the center of'),
 Document(metadata={'id': 'tr945181.png', 'chunk': 8}, page_content='of the image'),
 Document(metadata={'id': 'tr945181.png', 'chunk': 9}, page_content=', and the desert is'),
 Document(metadata={'id': 'tr945181.png', 'chunk': 10}, page_content='is a mix of red'),
 Document(metadata={'id'